In [6]:
import os
import cv2
import tkinter as tk
from tkinter import simpledialog
import mediapipe as mp
import matplotlib.pyplot as plt

# blank comment and added on more comment



import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import save_model



from tensorflow.keras.models import load_model


# 1.DATA COLLECTION UTPTO 1000 images per label
#### here we detct the input from users camera and save the images in the respective folder
#### also the part of image containing hand is saved by cropping it 
#### the only problem in this is with gestures requiring two hands
#### to solve this issue a diffrent approach need to be seen.

In [7]:
def cptr_crop(nm_img):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils
    
    
    fldName = "CollectedData"
    if not os.path.exists(fldName):
        os.makedirs(fldName)
    
    cap = cv2.VideoCapture(0)
    print("Press 's' to save the cropped hand image with label, or 'q' to quit.")
    
    img_cnt = 0
    
    while img_cnt < nm_img:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break
        
    
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rslt = hands.process(img_rgb)
        
    
        if rslt.multi_hand_landmarks:
            for hand_landmarks in rslt.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        
        cv2.imshow("Capture Image with Landmarks", frame)
        #caputerr the imae with label and landmarkkk
        
        
        
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('s') and img_cnt < nm_img:  
        
            root = tk.Tk()
            root.withdraw()
            label = simpledialog.askstring("Input", "Enter the label for this image:")
            root.destroy()
            
            
            # Take lanbel inpute from the user ....
            #The only problem is that for a single label user had to enter same text multiple time 
            # problem need to be considered i think it can be resolved using the previously used GUI interface ... handeled later
            
            
            if label and rslt.multi_hand_landmarks:
                
                label_dir = os.path.join(fldName, label)
                
                if not os.path.exists(label_dir):
                    os.makedirs(label_dir)
                
                for idx, hand_landmarks in enumerate(rslt.multi_hand_landmarks):
                    
                    img_h, img_w, _ = frame.shape
                    x_min, y_min = img_w, img_h
                    x_max, y_max = 0, 0
                    
                    for landmark in hand_landmarks.landmark:
                        x, y = int(landmark.x * img_w), int(landmark.y * img_h)
                        x_min, y_min = min(x, x_min), min(y, y_min)
                        x_max, y_max = max(x, x_max), max(y, y_max)
                    
                    
                    padding = 20
                    x_min = max(0, x_min - padding)
                    y_min = max(0, y_min - padding)
                    x_max = min(img_w, x_max + padding)
                    y_max = min(img_h, y_max + padding)
                    
                    
                    cropped_hand = frame[y_min:y_max, x_min:x_max]
                    
                    
                    file_count = len(os.listdir(label_dir)) + 1 
                    file_name = os.path.join(label_dir, f"{file_count}.jpg")
                    cv2.imwrite(file_name, cropped_hand)
                    print(f"Cropped hand image saved as {file_name}")
                    img_cnt += 1
                    
                    
                    if img_cnt >= nm_img:
                        break
            else:
                print("No hand detected or label not provided.")
        
        elif key == ord('q'):  
            break

    cap.release()
    cv2.destroyAllWindows()


cptr_crop(nm_img=100) 
# The only isssue with this part was saving multiple images with same label was not possible 
# so the thought was to make separate folder for each label and insert a multiple images in that folder 
# the labels will be later handeled using label encoder or any other technique 

# Code runnuhgg fine with no errors debugging not done

Press 's' to save the cropped hand image with label, or 'q' to quit.


# 2. Loading the data and using label encoder

In [3]:
def load_data(fldName):
    images = []
    labels = []
    
    for label in os.listdir(fldName):
        label_dir = os.path.join(fldName, label)
        for img_file in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_file)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Image at {img_path} is not valid and will be skipped.")
                continue
            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
            img = cv2.resize(img, (128, 128)) 
            images.append(img)
            labels.append(label)
    
    images = np.array(images) / 255.0 
    labels = np.array(labels)
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    np.save('label_class.npy', label_encoder.classes_)
    
    
    x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    
    return x_train, x_test, y_train, y_test

# LAbel encoder seems fine for now no error saved label of encoder so that we can revert back them to respective labels 

# 3. Building a MODEL

#### 3.1. CNN model (model1)

In [17]:
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(len(np.unique(y_train)), activation='softmax')  # Number of output classes
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


fldName = "CollectedData"
x_train, x_test, y_train, y_test = load_data(fldName)


model = build_model()


history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=32)
model_save_path = 'models/model1.h5'


model.save(model_save_path)

print(f"Model saved to {model_save_path}")

C:\Users\KIIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 12s 165ms/step - accuracy: 0.6597 - loss: 1.0721 - val_accuracy: 1.0000 - val_loss: 7.2988e-04
Epoch 2/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 12s 199ms/step - accuracy: 1.0000 - loss: 1.7362e-04 - val_accuracy: 1.0000 - val_loss: 2.5146e-05
Epoch 3/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 17s 277ms/step - accuracy: 1.0000 - loss: 1.2730e-05 - val_accuracy: 1.0000 - val_loss: 5.5702e-06
Epoch 4/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 17s 278ms/step - accuracy: 1.0000 - loss: 3.4112e-06 - val_accuracy: 1.0000 - val_loss: 2.7665e-06
Epoch 5/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 12s 194ms/step - accuracy: 1.0000 - loss: 1.8792e-06 - val_accuracy: 1.0000 - val_loss: 1.7310e-06
Epoch 6/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 12s 189ms/step - accuracy: 1.0000 - loss: 8.0353e-07 - val_accuracy: 1.0000 - val_loss: 1.1589e-06
Epoch 7/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 12s 191ms/step - accuracy: 1.0000 - loss: 6.0348e-07 - val_accuracy: 1.0000 - val_loss: 9.2683e-07
Epoch 8/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 12s 194

Model saved to models/model1.h5


#### 3.2 CNN 2 model(model 2)

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(y_train)), activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

fldName = "CollectedData"
x_train, x_test, y_train, y_test = load_data(fldName)


model = build_model()


history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=32)
model_save_path = 'models/model2.h5'


model.save(model_save_path)

print(f"Model saved to {model_save_path}")

C:\Users\KIIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 23s 332ms/step - accuracy: 0.6583 - loss: 1.0317 - val_accuracy: 0.9959 - val_loss: 0.0117
Epoch 2/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 22s 355ms/step - accuracy: 0.9991 - loss: 0.0032 - val_accuracy: 1.0000 - val_loss: 3.9803e-04
Epoch 3/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 30s 487ms/step - accuracy: 1.0000 - loss: 5.8194e-05 - val_accuracy: 1.0000 - val_loss: 1.0301e-05
Epoch 4/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 28s 448ms/step - accuracy: 1.0000 - loss: 7.1958e-06 - val_accuracy: 1.0000 - val_loss: 6.3803e-06
Epoch 5/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 47s 545ms/step - accuracy: 1.0000 - loss: 3.3674e-06 - val_accuracy: 1.0000 - val_loss: 4.7089e-06
Epoch 6/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 29s 460ms/step - accuracy: 1.0000 - loss: 3.1086e-06 - val_accuracy: 1.0000 - val_loss: 3.6370e-06
Epoch 7/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 19s 297ms/step - accuracy: 1.0000 - loss: 2.0958e-06 - val_accuracy: 1.0000 - val_loss: 2.9241e-06
Epoch 8/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 13s 204ms/step 

Model saved to models/model2.h5


#### 3.3 MobileNetV2(model 3)

In [23]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(len(np.unique(y_train)), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False  # Freeze base layers
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

fldName = "CollectedData"
x_train, x_test, y_train, y_test = load_data(fldName)

models_folder = "models"
if not os.path.exists(models_folder):
    os.makedirs(models_folder)

# Save the model
model_path = os.path.join(models_folder, "model3.h5")
model.save(model_path)
print(f"Model saved to {model_path}")

Model saved to models\model3.h5


# FINAL PREDICTION REAL TIME

In [5]:
model = load_model('models/model2.h5')
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('label_class.npy')  

# Loading the saved labels and saved model
# or else we would have to train it every time we use it 
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

def prdct_gstr():

    cap = cv2.VideoCapture(0)
    print("Press 'q' to quit.")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break
        
    
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rslt = hands.process(img_rgb)
        
        if rslt.multi_hand_landmarks:
            for hand_landmarks in rslt.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                
                img_h, img_w, _ = frame.shape
                x_min, y_min = img_w, img_h
                x_max, y_max = 0, 0
                for landmark in hand_landmarks.landmark:
                    x, y = int(landmark.x * img_w), int(landmark.y * img_h)
                    x_min, y_min = min(x, x_min), min(y, y_min)
                    x_max, y_max = max(x, x_max), max(y, y_max)

                padding = 20
                x_min = max(0, x_min - padding)
                y_min = max(0, y_min - padding)
                x_max = min(img_w, x_max + padding)
                y_max = min(img_h, y_max + padding)
                
                cropped_hand = frame[y_min:y_max, x_min:x_max]
                cropped_hand = cv2.cvtColor(cropped_hand, cv2.COLOR_BGR2RGB) 
                cropped_hand = cv2.resize(cropped_hand, (128, 128))  
                cropped_hand = np.expand_dims(cropped_hand, axis=0) / 255.0  
                
            
                predictions = model.predict(cropped_hand)
                gesture_label_encoded = np.argmax(predictions, axis=1)[0]
                gesture_label = label_encoder.inverse_transform([gesture_label_encoded])[0]
                
                
                cv2.putText(frame, f'Gesture: {gesture_label}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    
        cv2.imshow("Real-time Hand Gesture Prediction", frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):  # Quit
            break

    cap.release()
    cv2.destroyAllWindows()

prdct_gstr()

Press 'q' to quit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46